**LLM Based C++ Debugger**

In [ ]:
import openai
import subprocess
import os
import re


openai.api_key = "YOUR-OPENAI-API-KEY"

# MODEL_NAME = "gpt-3.5-turbo"  # or "gpt-4", etc.

In [ ]:
import time

In [ ]:

import subprocess
import re
import textwrap


###############################################################################
# 1) Define a function to compile code with -Wall -Werror
#    so warnings are also counted as errors.
###############################################################################
def compile_code(filename):
    """
    Compiles the given C++ file using g++ with -Wall -Werror flags.
    Returns a tuple (success, compiler_output):
      - success (bool): True if compilation succeeded with no warnings/errors
      - compiler_output (str): The combined stderr/stdout from the compiler
    """
    process = subprocess.run(
        ["g++", "-Wall", "-Werror", filename, "-o", "fixed_executable"],
        capture_output=True,
        text=True
    )
    success = (process.returncode == 0)
    compiler_output = (process.stderr + "\n" + process.stdout).strip()
    return success, compiler_output

###############################################################################
# 2) Parse *all* errors from the compiler output.
#    We'll collect them in the order they appear.
#    Then we fix the *last* one from that list.
###############################################################################
def parse_all_errors(compiler_output):
    """
    Looks for lines of the form:
      fixed_code.cpp:LINE:COL: error: ...
      fixed_code.cpp:LINE:COL: warning: ...
    Returns a list of tuples (line_number, entire_error_line).
    We'll keep them in the order found in the output.

    If you want to ignore warnings, you can filter out lines containing 'warning:'.
    """
    # We'll capture lines matching something like:
    # "fixed_code.cpp:123:5: error: <message>"
    # regex = r'(fixed_code\.cpp:(\d+):\d+:\s*(?:fatal error|error|warning):\s*.*)'
    regex = r'(fixed_code\.cpp:(\d+):\d+: (fatal error|error|warning): .*)'

    matches = re.findall(regex, compiler_output)
    errors = []
    for match in matches:
        # Ensure match contains all expected groups
        if len(match) == 3:
            full_line, line_str, _ = match  # Ignore the error type for now
            line_number = int(line_str)
            errors.append((line_number, full_line))

    return errors

###############################################################################
# 3) Ask the LLM to fix just the *last* error by giving the entire code
#    and the specific error message.
###############################################################################
def ask_llm_to_fix_code(current_code, error_line):
    """
    current_code: the entire current .cpp code as a string
    error_line: the error message string (e.g. "fixed_code.cpp:140:10: error: ...")

    Return the LLM's revised code (as a string).
    """
    prompt = textwrap.dedent(f"""
    You are a meticulous AI debugger.
    Below is the full C++ code, followed by a single compiler error or warning message.
    Please fix ONLY what is needed to address this error,
    and keep the rest of the code intact.

    --- FULL CODE ---
    {current_code}

    --- COMPILER ERROR/WARNING ---
    {error_line}

    Your output should be the entire corrected C++ code with minimal changes.
    """)

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # or gpt-4 if you have access
        messages=[
            {"role": "system", "content": "You are a careful programming assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    llm_output = response.choices[0].message.content

    # Remove any markdown fences:
    lines = llm_output.split('\n')
    cleaned = []
    for line in lines:
        if '```' in line:
            continue
        cleaned.append(line)
    fixed_code = "\n".join(cleaned).strip()

    return fixed_code

###############################################################################
# 4) Main iterative routine:
#    - Read original code from test1.cpp
#    - Compile
#    - Parse all errors
#    - Fix the *last* error
#    - Repeat
###############################################################################
def iterative_fix_bottom_up(filename="test1.cpp", max_iterations=10):
    """
    Reads buggy code from filename, attempts to fix
    compilation errors in a bottom-up manner (starting from the
    last error in the compiler's output).
    """
    # Read entire code into memory
    with open(filename, "r") as f:
        code_lines = f.read()

    current_code = code_lines

    for iteration in range(1, max_iterations + 1):
        print(f"\n=== Iteration #{iteration} ===")

        # Write current_code to "fixed_code.cpp"
        with open("fixed_code.cpp", "w") as f:
            f.write(current_code)

        # Compile
        success, compiler_out = compile_code("fixed_code.cpp")

        if success:
            print("Compilation succeeded with -Wall -Werror (no warnings/no errors)!")
            print("You can run your program via: !./fixed_executable")
            break
        else:
            print("Compiler output:\n")
            print(compiler_out)

            # Parse all errors
            all_errors = parse_all_errors(compiler_out)

            if not all_errors:
                print("No recognized errors/warnings found, but compilation failed.")
                print("Possibly a link error or a different message. Stopping.")
                break

            # We'll fix the *last* error in the list
            # The last element in all_errors is (line_number, entire_error_line)
            line_num, error_line = all_errors[-1]

            print(f"\nFixing the last error from the stack: line {line_num}")
            print(f"Error message: {error_line}")

            # Ask LLM to fix
            fixed_code = ask_llm_to_fix_code(current_code, error_line)

            if fixed_code.strip() == current_code.strip():
                print("LLM did not change the code. Potentially stuck. Stopping.")
                break
            else:
                # Update current_code
                current_code = fixed_code
    else:
        print("\nReached max iterations without a clean compile. Please inspect manually.")

###############################################################################
# 7) Run the iterative fix
###############################################################################
iterative_fix_bottom_up("test2.cpp", max_iterations=10)




=== Iteration #1 ===
Compiler output:

fixed_code.cpp:4:10: fatal error: cmaths: No such file or directory
    4 | #include <cmaths>     // For mathematical functions like cos, sin
      |          ^~~~~~~~
compilation terminated.

Fixing the last error from the stack: line 4
Error message: fixed_code.cpp:4:10: fatal error: cmaths: No such file or directory

=== Iteration #2 ===
Compilation succeeded with -Wall -Werror (no warnings/no errors)!
You can run your program via: !./fixed_executable


In [ ]:
!g++ fixed_code.cpp -o fixed_code

In [ ]:
!g++ test2Copy.cpp -o fixed_code